# **Python PostgreSQL Database Connectivity**


## **1. Installing Required Libraries**

To connect to PostgreSQL from Python, we need the `psycopg2` library. It is the most popular PostgreSQL adapter for Python.

### **1.1 Install psycopg2**
Run the following command to install `psycopg2`:
```bash
pip install psycopg2
```

If you encounter issues installing `psycopg2`, you can use the binary version:
```bash
pip install psycopg2-binary
```

### **1.2 Verify Installation**
```python
import psycopg2
print("psycopg2 installed successfully!")
```


## **2. Connecting to a PostgreSQL Database**

### **2.1 Import the Library**
```python
import psycopg2
```

### **2.2 Establish a Connection**
To connect to a PostgreSQL database, you need the following details:
- **Database name**
- **User name**
- **Password**
- **Host** (default is `localhost`)
- **Port** (default is `5432`)

```python
# Database connection parameters
db_config = {
    'dbname': 'database_name',
    'user': 'your_username',
    'password': 'your_password',
    'host': 'localhost',
    'port': '5432'
}

# Establish a connection
try:
    connection = psycopg2.connect(**db_config)
    print("Connection established successfully!")
except Exception as e:
    print(f"Error: {e}")
```

### **2.3 Connection Parameters**
- **dbname**: The name of the database you want to connect to.
- **user**: The username used to authenticate.
- **password**: The password used to authenticate.
- **host**: The database server address (default is `localhost`).
- **port**: The port number to connect to (default is `5432`).

## **3. Executing SQL Queries**

### **3.1 Create a Cursor**
A cursor allows you to execute SQL queries and fetch results.

```python
cursor = connection.cursor()
```

### **3.2 Execute a Query**
You can execute SQL queries using the `execute()` method.

#### **Example: Fetch All Customers**
```python
# Fetch all customers
cursor.execute("SELECT * FROM customers;")
customers = cursor.fetchall()

# Print the results
for customer in customers:
    print(customer)
```

#### **Example: Insert a New Customer**
```python
# Insert a new customer
insert_query = """
INSERT INTO customers (customer_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state)
VALUES (%s, %s, %s, %s, %s);
"""
data = ('new_customer_id', 'new_unique_id', '12345', 'São Paulo', 'SP')

cursor.execute(insert_query, data)
connection.commit()  # Commit the transaction
print("Customer inserted successfully!")
```

### **3.3 Fetching Data**
- **fetchone()**: Fetches the next row of a query result set.
- **fetchall()**: Fetches all rows of a query result set.
- **fetchmany(size)**: Fetches the next set of rows of a query result set.

```python
# Fetch one row
row = cursor.fetchone()
print(row)

# Fetch many rows
rows = cursor.fetchmany(5)
for row in rows:
    print(row)
```

## **4. Handling Transactions**

### **4.1 What is a Transaction?**
A transaction is a sequence of operations performed as a single logical unit of work. It ensures **atomicity**, **consistency**, **isolation**, and **durability** (ACID properties).

### **4.2 Example: Using Transactions**
```python
try:
    # Start a transaction
    cursor.execute("BEGIN;")

    # Insert a new order
    insert_order_query = """
    INSERT INTO orders (order_id, customer_id, order_status)
    VALUES (%s, %s, %s);
    """
    order_data = ('new_order_id', 'some_customer_id', 'pending')
    cursor.execute(insert_order_query, order_data)

    # Insert a payment record
    insert_payment_query = """
    INSERT INTO payments (order_id, payment_sequential, payment_type, payment_installments, payment_value)
    VALUES (%s, %s, %s, %s, %s);
    """
    payment_data = ('new_order_id', 1, 'credit_card', 1, 100.00)
    cursor.execute(insert_payment_query, payment_data)

    # Commit the transaction
    connection.commit()
    print("Transaction completed successfully!")
except Exception as e:
    # Rollback in case of error
    connection.rollback()
    print(f"Error: {e}")
```

## **5. Using Connection Pooling**

### **5.1 What is Connection Pooling?**
Connection pooling allows you to reuse database connections, reducing the overhead of creating and closing connections repeatedly.

### **5.2 Install psycopg2 Pool**
```bash
pip install psycopg2-pool
```

### **5.3 Example: Using Connection Pooling**
```python
from psycopg2_pool import SimpleConnectionPool

# Create a connection pool
pool = SimpleConnectionPool(
    minconn=1,
    maxconn=10,
    **db_config
)

# Get a connection from the pool
connection = pool.getconn()

# Use the connection
cursor = connection.cursor()
cursor.execute("SELECT * FROM customers;")
customers = cursor.fetchall()
for customer in customers:
    print(customer)

# Release the connection back to the pool
pool.putconn(connection)
```

## **6. Error Handling**

### **6.1 Handling Database Errors**
Always handle database errors gracefully using `try-except` blocks.

```python
try:
    cursor.execute("SELECT * FROM non_existent_table;")
except psycopg2.Error as e:
    print(f"Database error: {e}")
```

### **6.2 Common Errors**
- **OperationalError**: Issues with the database connection.
- **IntegrityError**: Violation of database constraints (e.g., duplicate primary key).
- **ProgrammingError**: Syntax errors in SQL queries.

## **7. Best Practices for Database Connectivity**

### **7.1 Use Context Managers**
Use the `with` statement to automatically handle connection and cursor cleanup.

```python
with psycopg2.connect(**db_config) as connection:
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM customers;")
        customers = cursor.fetchall()
        for customer in customers:
            print(customer)
```

### **7.2 Parameterize Queries**
Always use parameterized queries to prevent SQL injection.

```python
cursor.execute("SELECT * FROM customers WHERE customer_id = %s;", ('some_customer_id',))
```

### **7.3 Close Connections**
Always close connections and cursors when done.

```python
cursor.close()
connection.close()
```

### **7.4 Use Connection Pooling**
For applications with high database traffic, use connection pooling to improve performance.

## **8. Advanced Topics**

### **8.1 Using JSON Data**
PostgreSQL supports storing and querying JSON data. You can use `JSONB` for efficient querying.

```python
# Add a JSONB column to the customers table
cursor.execute("ALTER TABLE customers ADD COLUMN preferences JSONB;")

# Insert JSON data
cursor.execute("""
UPDATE customers
SET preferences = '{"newsletter": true, "language": "Portuguese"}'
WHERE customer_id = 'some_customer_id';
""")
connection.commit()
```

### **8.2 Using Stored Procedures**
You can call PostgreSQL stored procedures from Python.

```python
# Create a stored procedure
cursor.execute("""
CREATE OR REPLACE FUNCTION calculate_seller_revenue(seller_id VARCHAR)
RETURNS NUMERIC AS $$
DECLARE
    total_revenue NUMERIC;
BEGIN
    SELECT SUM(price) INTO total_revenue
    FROM order_items
    WHERE seller_id = seller_id;

    RETURN total_revenue;
END;
$$ LANGUAGE plpgsql;
""")
connection.commit()

# Call the stored procedure
cursor.callproc('calculate_seller_revenue', ('some_seller_id',))
result = cursor.fetchone()
print(f"Total Revenue: {result[0]}")
```

## **9. Real-World Example: Building a Customer Analytics Dashboard**

### **9.1 Problem Statement**
We want to build a customer analytics dashboard that:
- Displays the total number of customers.
- Shows the top 5 customers by total spending.
- Tracks the number of orders by status.

### **9.2 Solution**
We will write Python code to fetch and display this data.

```python
# Fetch total number of customers
cursor.execute("SELECT COUNT(*) FROM customers;")
total_customers = cursor.fetchone()[0]
print(f"Total Customers: {total_customers}")

# Fetch top 5 customers by total spending
cursor.execute("""
SELECT c.customer_id, c.customer_city, SUM(p.payment_value) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN payments p ON o.order_id = p.order_id
GROUP BY c.customer_id, c.customer_city
ORDER BY total_spent DESC
LIMIT 5;
""")
top_customers = cursor.fetchall()
print("Top 5 Customers by Spending:")
for customer in top_customers:
    print(customer)

# Fetch number of orders by status
cursor.execute("""
SELECT order_status, COUNT(*) AS order_count
FROM orders
GROUP BY order_status;
""")
order_status_counts = cursor.fetchall()
print("Orders by Status:")
for status, count in order_status_counts:
    print(f"{status}: {count}")
```